# CLD Conversion

This script takes the output of a chemiluminescence monitor and converts it to a for ruby usable format.
We expect two columns. One denoted "timestamp" containting the Date and Time of the Measuremnt and another denoted "no" conatining the measured NO concentration (in ppb). 

Next to the mere semantic update the script can make sure to have a timezone set and it can apply a calibration function to the raw NO values.

In [1]:
require 'narray'
require 'daru'
include Daru
nil

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

## Variables

* prefix: Array of prefixes of the files that are to be converted. The files need to be in the same directory
* dir: Directory containing the files

* change_tz: If false, does not touch timestamp at all. If true it sets a timezone.
* input_tz: a string that contains the timezone in which the data was recorded. The format should be "+07:00".
* output_tz: The timezone you want your data file to be in afterward. We use the ruby DateTime object for encoding so the format is timezone (as float) divided by 24.

* apply_calibration: The no concentrations of the CLD are not in ppb (although it says so), but need to be calibrated. If the variable is set to true a calibration function is applied to the data.

In [2]:
# String or Array of Strings with prefix of raw cld files
prefix = ["20160121_calib", "20160121", "20160122", "20160125", "20160219", "20160220", "20160221", "20160222"] 

dir = "../CLD/"


# Change Time Zone
# Brings Time Zone in Standard DateTime format and sets the TimeZone
change_tz = true
input_tz = "+01:00"
output_tz = 1.0/24

apply_calibration = false



The variables below should be good as they are. They are in accordance with the naming convention.

* raw_suffix: The raw CLD data has a special suffix, that the converted fiel won't have. It is specified here.
* filetype: The script expects a csv-stile file. If you want to give it another suffix you can change it here.

In [ ]:
raw_suffix = "_raw"
filetype = ".csv"
nil

## Calibration function
Update this function if you want to apply another calibration. It has to work with a float and should return a float

In [3]:
def calib(no) 
  (no + 1.34614)/1.70233
end
nil

## Setup

In [4]:
if prefix.is_a?(String)
  prefix = [prefix]
end

unless prefix.is_a?(Array)
  raise "prefix needs to be a String or Array of Strings, but is a #{prefix.class}"
end
nil

## Conversion
Status at the bottom.

In [6]:
start = DateTime.now

prefix.each do |p| 
  unless p.is_a?(String)
    raise "prefix needs to be an Array of Strings, but contains a #{p.class}."
  end
  
  input = dir + p + raw_suffix+ filetype
  output = dir + p + filetype
  
  d = DataFrame.from_csv(input)
  
  unless d.vectors.to_a.include?("no")
    raise "header needs to contain 'no'"
  end
  
  unless d.vectors.to_a.include?("timestamp")
    raise "header need to containt 'timestamp'"
  end
  
  d.recode_rows do |row|
    
    no = row["no"].to_s.gsub(",", ".").to_f
    
    if apply_calibration
      row["no"] = calib(no)
    else
      row["no"] = no
    end
    row["timestamp"] =  DateTime.parse(row["timestamp"] + input_tz).new_offset(output_tz)
    row
  end
  
  d.write_csv(output)
  
  puts "Wrote #{output}"
  
end

stop = DateTime.now
delta = (stop - start).to_f*24*60

puts "Done. It took #{delta} minutes."
nil

Written ../CLD/20160121_calib.csv
Written ../CLD/20160121.csv
Written ../CLD/20160122.csv
Written ../CLD/20160125.csv
Written ../CLD/20160219.csv
Written ../CLD/20160220.csv
Written ../CLD/20160221.csv
Written ../CLD/20160222.csv
Done. It took 3.5901217612833336 minutes.
